In [4]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('cardiovascular.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - loss: 2259.6401 - val_loss: 1008.3593
Epoch 2/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - loss: 1269.7672 - val_loss: 1029.0723
Epoch 3/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - loss: 864.6291 - val_loss: 1085.9209
Epoch 4/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - loss: 901.7354 - val_loss: 1008.6356
Epoch 5/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - loss: 877.7940 - val_loss: 953.9826
Epoch 6/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - loss: 1048.7140 - val_loss: 962.8977
Epoch 7/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 874.5490 - val_loss: 986.5991
Epoch 8/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - loss: 794.8610 - val_loss: 1211.4044
Epoch 9/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - loss: 927.1743 - val_loss: 921.4501
Epoch 10/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - loss: 913.4915 - val_loss: 995.6284
Epoch 11/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - los

135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 667.8352 - val_loss: 1029.8215
Epoch 65/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - loss: 601.6699 - val_loss: 1160.7582
Epoch 66/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - loss: 617.1469 - val_loss: 1121.2935
Epoch 67/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 565.5302 - val_loss: 1164.7025
Epoch 68/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - loss: 661.7471 - val_loss: 1201.3033
Epoch 69/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - loss: 559.5917 - val_loss: 1164.8514
Epoch 70/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - loss: 600.1512 - val_loss: 1154.4432
Epoch 71/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - loss: 596.6898 - val_loss: 1080.2354
Epoch 72/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - loss: 640.8420 - val_loss: 1010.2465
Epoch 73/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - loss: 656.5444 - val_loss: 1193.9955
Epoch 74/100
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss:

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
                                              Product  Forecasted Quantity
25               Losartan 100mg (Natrasol) 100&#039;s            66.969505
28          Losartan K (Natrasol) 50mg Cap 100&#039;s            38.873123
41                              RM  Losartan 50mg Tab            27.142553
0          Amlodipine 10mg (Diadipine) Tab 100&#039;s            24.813599
1            Amlodipine 10mg (Lodipex) Tab 100&#039;s            21.583229
3           Amlodipine 5mg (Diadipine) Tab 100&#039;s            20.990294
4            Amlodipine 5mg (Johnvasc) Tab 100&#039;s            20.707180
26               Losartan 100mg (Natrazol) 100&#039;s            18.700436
24                 Losartan 100mg (Losaar) 100&#039;s            17.009441
19                          Isordil 5mg Subligual Tab            16.884552
42                         